# <center>(AI4LR) - Notebook 6 [Final]
    
Total Points: 100

This notebook is designed to get you fimiliar with the U-Net architecture. (and choosing a backbone)

### **NOTE: Open this notebook in kaggle and import Artificial Lunar Landscape Dataset.** 

## > If you run this notebook as it is, you will get the val_iou_score of around 0.20 (remember to use GPU for training the model)

## > Your goal is to increase the val_iou_score as much as you can for this project using any method. The evaluation of this assignment will be based on your acquired val_iou_score. One point for each increasing 0.01 val_iou_score. 

> For example - if val_iou_score = 0.41, your points will be 41/100. 

### Some tips to increase the performance
* Increase the number of epochs
* Increase the number of layers in your model
* Using SOTA high performance networks with transfer learning
* Using callbacks and carefully observing your model performance

You are free to use other techniques too. 

In [ ]:
!pip install segmentation_models

In [ ]:
import os
import numpy as np
from matplotlib import pyplot as plt

import cv2
import keras
from tqdm import tqdm

import tensorflow as tf
import glob
from PIL import Image
from sklearn.model_selection import train_test_split

from skimage.io import imread
from skimage.transform import resize
import numpy as np
import math
from tensorflow.keras.utils import to_categorical, Sequence

import segmentation_models as sm

import datetime

* Provide environment variable SM_FRAMEWORK=keras / SM_FRAMEWORK=tf.keras before import segmentation_models
* Change framework sm.set_framework('keras') / sm.set_framework('tf.keras')

In [ ]:
os.environ["SM_FRAMEWORK"] = "tf.keras"
sm.set_framework('tf.keras')
keras.backend.set_image_data_format('channels_last')

## Data Preprocessing Pipeline 

In [ ]:
H = 256 # height of image
W = 256 # width of image

'''This function is used to return the list of path for images and masks in
sorted order from the given directory respectively.'''
# function to return list of image paths and mask paths 
def process_data(IMG_DIR, MASK_DIR):
    images = [os.path.join(IMG_DIR, x) for x in sorted(os.listdir(IMG_DIR))]
    masks = [os.path.join(MASK_DIR, x) for x in sorted(os.listdir(MASK_DIR))]

    return images, masks

'''This function is used to return splitted list of images and corresponding 
mask paths in train and test by providing test size.'''
# function to load data and train test split
def load_data(IMG_DIR, MASK_DIR):
    X, y = process_data(IMG_DIR, MASK_DIR)
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42)
     
    return X_train, X_test, y_train, y_test

'''This function is used to read images. It takes image path as input. 
After reading image it is resized by width and height provide above(256 x 256). 
Next normalization is done by dividing each values with 255. And the result is returned.'''
# function to read image
def read_image(x):
    x = cv2.imread(x, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W, H))
    x = x / 255.0
    x = x.astype(np.float32)
    return x

'''This function is used to read masks.'''
# function to read mask
def read_mask(x):
    x = cv2.imread(x, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (W, H))
    x = x.astype(np.int32)
    return x

'''This function is used to generate tensorflow data pipeline. 
The tensorflow data pipeline is mapped to function ‘preprocess’ .'''
# function for tensorflow dataset pipeline
def tf_dataset(x, y, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.shuffle(buffer_size=5000)
    dataset = dataset.map(preprocess)
    dataset = dataset.batch(batch)
    dataset = dataset.repeat()
    dataset = dataset.prefetch(2)
     return dataset

'''This function takes image and mask path. 
It reads the image and mask as provided by paths. 
Mask is one hot encoded for multi class segmentation (here 4 class).'''
# function to read image and mask amd create one hot encoding for mask
def preprocess(x, y):
    def f(x, y):
        x = x.decode()
        y = y.decode()

        image = read_image(x)
        mask = read_mask(y)

        return image, mask

    image, mask = tf.numpy_function(f, [x, y], [tf.float32, tf.int32])
    mask = tf.one_hot(mask, 4, dtype=tf.int32)
    image.set_shape([H, W, 3])
    mask.set_shape([H, W, 4])

    return image, mask

## Load the dataset

In [ ]:
'''RENDER_IMAGE_DIR_PATH: ‘Path of image directory’
GROUND_MASK_DIR_PATH: ‘Path of mask directory’

Here load_data function is called. This will load the dataset paths and 
split it into X_train, X_test, y_train, y_test '''

RENDER_IMAGE_DIR_PATH = '../input/artificial-lunar-rocky-landscape-dataset/images/render'
GROUND_MASK_DIR_PATH = '../input/artificial-lunar-rocky-landscape-dataset/images/clean'

X_train, X_test, y_train, y_test = load_data(RENDER_IMAGE_DIR_PATH, GROUND_MASK_DIR_PATH)
print(f"Dataset:\n Train: {len(X_train)} \n Test: {len(X_test)}")

## Generate tensorflow data pipeline

In [ ]:
batch_size = 8

'''Here the tf_dataset function is called will generate the tensorflow data pipeline.'''
# calling tf_dataset
train_dataset = tf_dataset(X_train, y_train, batch=batch_size)
valid_dataset = tf_dataset(X_test, y_test, batch=batch_size)

## Creating U-net Architecture

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPool2D, UpSampling2D, Concatenate
from tensorflow.keras.models import Model

'''conv_block it is used to create one block with two convolution layer 
followed by BatchNormalization and activation function relu. 
If the pooling is required then Maxpool2D is applied and return it else not.'''
# function to create convolution block
def conv_block(inputs, filters, pool=True):
    x = Conv2D(filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    if pool == True:
        p = MaxPool2D((2, 2))(x)
        return x, p
    else:
        return x

'''build_unet it is used to create the U-net architecture.'''
# function to build U-net
def build_unet(shape, num_classes):
    inputs = Input(shape)

    """ Encoder """
    x1, p1 = conv_block(inputs, 16, pool=True)
    x2, p2 = conv_block(p1, 32, pool=True)
    x3, p3 = conv_block(p2, 48, pool=True)
    x4, p4 = conv_block(p3, 64, pool=True)
    
     """ Bridge """
    b1 = conv_block(p4, 128, pool=False)

    """ Decoder """
    u1 = UpSampling2D((2, 2), interpolation="bilinear")(b1)
    c1 = Concatenate()([u1, x4])
    x5 = conv_block(c1, 64, pool=False)

    u2 = UpSampling2D((2, 2), interpolation="bilinear")(x5)
    c2 = Concatenate()([u2, x3])
    x6 = conv_block(c2, 48, pool=False)

    u3 = UpSampling2D((2, 2), interpolation="bilinear")(x6)
    c3 = Concatenate()([u3, x2])
    x7 = conv_block(c3, 32, pool=False)

    u4 = UpSampling2D((2, 2), interpolation="bilinear")(x7)
    c4 = Concatenate()([u4, x1])
    x8 = conv_block(c4, 16, pool=False)

    """ Output layer """
    output = Conv2D(num_classes, 1, padding="same", activation="softmax")(x8)

    return Model(inputs, output)

In [ ]:
# calling build_unet function
model = build_unet((256, 256, 3), 4)
model.summary()

## Load model and compile

In [ ]:
# importing libraries
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, TensorBoard
from segmentation_models.metrics import iou_score
import datetime, os

""" Hyperparameters """
img_shape = (256, 256, 3)
num_classes = 4
lr = 1e-4
batch_size = 16
epochs = 5

""" Model """
model = build_unet(img_shape, num_classes)
model.compile(loss="categorical_crossentropy", 
              optimizer=tf.keras.optimizers.Adam(lr), 
              metrics=[iou_score])


train_steps = len(X_train)//batch_size
valid_steps = len(X_test)//batch_size

## Train model 

In [ ]:
'''model.fit is used to train the model'''
model_history = model.fit(train_dataset,
        steps_per_epoch=train_steps,
        validation_data=valid_dataset,
        validation_steps=valid_steps,
        epochs=epochs,
    )

## Improved Method

## Data Preprocessing Pipeline

In [ ]:
img_dir = '../input/artificial-lunar-rocky-landscape-dataset/images/render'
mask_dir = '../input/artificial-lunar-rocky-landscape-dataset/images/clean'

images = [os.path.join(img_dir, x) for x in sorted(os.listdir(img_dir))]
masks = [os.path.join(mask_dir, x) for x in sorted(os.listdir(mask_dir))]

X_train = images[:8000]
y_train = masks[:8000]

X_valid = images[8000:]
y_valid = masks[8000:]

## Load the dataset

In [ ]:
class LunarDataset(Sequence):

    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        count = 0
        xtr = np.zeros((16, 480, 480, 3))
        for filename in batch_x:
            img = imread(filename)[:480, :480, :] / 255.0
            img = img.astype(np.float32)
            xtr[count] = img
            count += 1
            
        count = 0
        ytr = np.zeros((16, 480, 480, 4))
        for filename in batch_y:
            mask = imread(filename, as_gray = True)[:480, :480] // 0.07
            mask[mask == 3] = 2
            mask[mask == 10] = 3
         
            mask = to_categorical(mask, num_classes = 4)
            ytr[count] = mask
            count += 1

        return xtr, ytr.astype(np.int32)

train_dataset = LunarDataset(X_train, y_train, 16)
valid_dataset = LunarDataset(X_valid, y_valid, 16)

## Generate tensorflow data pipeline

In [ ]:
IMG_SHAPE = (480, 480, 3)

base_model = tf.keras.applications.InceptionV3(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

base_model.trainable = False

base_model.summary()

inputs = tf.keras.Input(shape=(480, 480, 3))

x = base_model(inputs, training=False)

x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(2)(x)

model = tf.keras.Model(inputs, outputs)

model.summary()

## Creating U-net Architecture

In [ ]:
BACKBONE = 'vgg16'
input_shape = (480, 480, 3)
n_classes = 4
activation = 'softmax'

model = sm.Unet(backbone_name = BACKBONE, 
                input_shape = input_shape, 
                classes = n_classes, 
                activation = activation,
                encoder_weights = 'imagenet')
model.summary()

## Load model and compile

In [ ]:
from keras.callbacks import ModelCheckpoint
lr = 1e-4
batch_size = 16
epochs = 17

metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

model.compile(loss = 'categorical_crossentropy', 
              optimizer = tf.keras.optimizers.Adam(lr), 
              metrics = metrics)

train_steps = len(X_train)//batch_size
valid_steps = len(X_valid)//batch_size

current_datetime = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

callbacks = [
        tf.keras.callbacks.ModelCheckpoint(filepath=f'models/lunarModel_{current_datetime}.h5',
                        monitor='val_iou_score', verbose=0, 
                        mode='max', save_best_model=True),
             
        tf.keras.callbacks.ReduceLROnPlateau(monitor="val_iou_score", mode='max', patience=4,
                          factor=0.1, verbose=0, min_lr=1e-6),
             
        tf.keras.callbacks.EarlyStopping(monitor="val_iou_score", patience=5, verbose=0, mode='max'),

        tf.keras.callbacks.TensorBoard(f'models/logs_{current_datetime}')
    ]

## Train model

In [ ]:
model_history = model.fit(train_dataset,
        steps_per_epoch=train_steps,
        validation_data=valid_dataset,
        validation_steps=valid_steps,
        epochs=epochs,
        callbacks=callbacks
    )

## [IMPORTANT] final model training history 
NOTE: If we find that your actual model score and what you paste here is differing, your assignment will get rejected.  

here ----

Results with VGG16 - 17 Epochs

Epoch 1/17
  1/500 [..............................] - ETA: 3:08:29 - loss: 1.6735 - iou_score: 0.0323 - f1-score: 0.0618
  2/500 [..............................] - ETA: 9:07 - loss: 1.6341 - iou_score: 0.0326 - f1-score: 0.0624   
500/500 [==============================] - 760s 1s/step - loss: 0.3566 - iou_score: 0.5339 - f1-score: 0.6016 - val_loss: 0.1722 - val_iou_score: 0.6035 - val_f1-score: 0.6817
Epoch 2/17
500/500 [==============================] - 626s 1s/step - loss: 0.1398 - iou_score: 0.7121 - f1-score: 0.8005 - val_loss: 0.1349 - val_iou_score: 0.6683 - val_f1-score: 0.7495
Epoch 3/17
500/500 [==============================] - 622s 1s/step - loss: 0.1150 - iou_score: 0.7542 - f1-score: 0.8385 - val_loss: 0.1111 - val_iou_score: 0.7470 - val_f1-score: 0.8320
Epoch 4/17
500/500 [==============================] - 624s 1s/step - loss: 0.1035 - iou_score: 0.7781 - f1-score: 0.8580 - val_loss: 0.0971 - val_iou_score: 0.7954 - val_f1-score: 0.8718
Epoch 5/17
500/500 [==============================] - 627s 1s/step - loss: 0.0967 - iou_score: 0.7913 - f1-score: 0.8681 - val_loss: 0.0895 - val_iou_score: 0.8057 - val_f1-score: 0.8793
Epoch 6/17
500/500 [==============================] - 589s 1s/step - loss: 0.0911 - iou_score: 0.8057 - f1-score: 0.8792 - val_loss: 0.0979 - val_iou_score: 0.7851 - val_f1-score: 0.8632
Epoch 7/17
500/500 [==============================] - 565s 1s/step - loss: 0.0884 - iou_score: 0.8094 - f1-score: 0.8821 - val_loss: 0.0919 - val_iou_score: 0.8000 - val_f1-score: 0.8754
Epoch 8/17
500/500 [==============================] - 621s 1s/step - loss: 0.0860 - iou_score: 0.8134 - f1-score: 0.8851 - val_loss: 0.0847 - val_iou_score: 0.8174 - val_f1-score: 0.8875
Epoch 9/17
500/500 [==============================] - 572s 1s/step - loss: 0.0826 - iou_score: 0.8225 - f1-score: 0.8918 - val_loss: 0.0844 - val_iou_score: 0.8146 - val_f1-score: 0.8854
Epoch 10/17
500/500 [==============================] - 572s 1s/step - loss: 0.0779 - iou_score: 0.8334 - f1-score: 0.8997 - val_loss: 0.0947 - val_iou_score: 0.7923 - val_f1-score: 0.8692
Epoch 11/17
500/500 [==============================] - 571s 1s/step - loss: 0.0787 - iou_score: 0.8286 - f1-score: 0.8961 - val_loss: 0.0933 - val_iou_score: 0.7920 - val_f1-score: 0.8677
Epoch 12/17
500/500 [==============================] - 626s 1s/step - loss: 0.0774 - iou_score: 0.8325 - f1-score: 0.8988 - val_loss: 0.0936 - val_iou_score: 0.7866 - val_f1-score: 0.8642
Epoch 13/17
500/500 [==============================] - 573s 1s/step - loss: 0.0661 - iou_score: 0.8567 - f1-score: 0.9160 - val_loss: 0.0777 - val_iou_score: 0.8356 - val_f1-score: 0.9007
Epoch 14/17
500/500 [==============================] - 627s 1s/step - loss: 0.0637 - iou_score: 0.8601 - f1-score: 0.9183 - val_loss: 0.0778 - val_iou_score: 0.8375 - val_f1-score: 0.9021
Epoch 15/17
500/500 [==============================] - 573s 1s/step - loss: 0.0622 - iou_score: 0.8614 - f1-score: 0.9191 - val_loss: 0.0781 - val_iou_score: 0.8387 - val_f1-score: 0.9029
Epoch 16/17
500/500 [==============================] - 629s 1s/step - loss: 0.0608 - iou_score: 0.8628 - f1-score: 0.9200 - val_loss: 0.0786 - val_iou_score: 0.8387 - val_f1-score: 0.9030
Epoch 17/17
500/500 [==============================] - 542s 1s/step - loss: 0.0592 - iou_score: 0.8642 - f1-score: 0.9210 - val_loss: 0.0792 - val_iou_score: 0.8348 - val_f1-score: 0.9002










Results with InceptionV3 - 17 Epochs


Epoch 1/17
  1/500 [..............................] - ETA: 3:07:30 - loss: 1.4688 - iou_score: 0.0228 - f1-score: 0.0440
  2/500 [..............................] - ETA: 8:56 - loss: 1.4301 - iou_score: 0.0221 - f1-score: 0.0428 
500/500 [==============================] - 672s 1s/step - loss: 0.3617 - iou_score: 0.4389 - f1-score: 0.4633 - val_loss: 0.1881 - val_iou_score: 0.5016 - val_f1-score: 0.5300
Epoch 2/17
500/500 [==============================] - 537s 1s/step - loss: 0.1563 - iou_score: 0.5796 - f1-score: 0.6418 - val_loss: 0.1390 - val_iou_score: 0.6709 - val_f1-score: 0.7595
Epoch 3/17
500/500 [==============================] - 536s 1s/step - loss: 0.1170 - iou_score: 0.7403 - f1-score: 0.8260 - val_loss: 0.1061 - val_iou_score: 0.7429 - val_f1-score: 0.8277
Epoch 4/17
500/500 [==============================] - 533s 1s/step - loss: 0.1054 - iou_score: 0.7699 - f1-score: 0.8514 - val_loss: 0.0963 - val_iou_score: 0.7836 - val_f1-score: 0.8620
Epoch 5/17
500/500 [==============================] - 597s 1s/step - loss: 0.0972 - iou_score: 0.7885 - f1-score: 0.8663 - val_loss: 0.0926 - val_iou_score: 0.7918 - val_f1-score: 0.8683
Epoch 6/17
500/500 [==============================] - 510s 1s/step - loss: 0.0929 - iou_score: 0.7985 - f1-score: 0.8736 - val_loss: 0.0857 - val_iou_score: 0.8097 - val_f1-score: 0.8823
Epoch 7/17
500/500 [==============================] - 536s 1s/step - loss: 0.0874 - iou_score: 0.8114 - f1-score: 0.8838 - val_loss: 0.1270 - val_iou_score: 0.7359 - val_f1-score: 0.8186
Epoch 8/17
500/500 [==============================] - 539s 1s/step - loss: 0.0848 - iou_score: 0.8175 - f1-score: 0.8882 - val_loss: 0.0888 - val_iou_score: 0.8083 - val_f1-score: 0.8808
Epoch 9/17
500/500 [==============================] - 535s 1s/step - loss: 0.0816 - iou_score: 0.8238 - f1-score: 0.8927 - val_loss: 0.1555 - val_iou_score: 0.7029 - val_f1-score: 0.7875
Epoch 10/17
500/500 [==============================] - 533s 1s/step - loss: 0.0806 - iou_score: 0.8266 - f1-score: 0.8948 - val_loss: 0.0955 - val_iou_score: 0.7862 - val_f1-score: 0.8637
Epoch 11/17
500/500 [==============================] - 596s 1s/step - loss: 0.0721 - iou_score: 0.8444 - f1-score: 0.9076 - val_loss: 0.0795 - val_iou_score: 0.8269 - val_f1-score: 0.8947
Epoch 12/17
500/500 [==============================] - 536s 1s/step - loss: 0.0684 - iou_score: 0.8516 - f1-score: 0.9126 - val_loss: 0.0790 - val_iou_score: 0.8297 - val_f1-score: 0.8966
Epoch 13/17
500/500 [==============================] - 534s 1s/step - loss: 0.0669 - iou_score: 0.8545 - f1-score: 0.9145 - val_loss: 0.0791 - val_iou_score: 0.8322 - val_f1-score: 0.8985
Epoch 14/17
500/500 [==============================] - 509s 1s/step - loss: 0.0655 - iou_score: 0.8565 - f1-score: 0.9159 - val_loss: 0.0788 - val_iou_score: 0.8315 - val_f1-score: 0.8980
Epoch 15/17
500/500 [==============================] - 530s 1s/step - loss: 0.0640 - iou_score: 0.8579 - f1-score: 0.9168 - val_loss: 0.0787 - val_iou_score: 0.8306 - val_f1-score: 0.8973
Epoch 16/17
500/500 [==============================] - 599s 1s/step - loss: 0.0629 - iou_score: 0.8596 - f1-score: 0.9179 - val_loss: 0.0804 - val_iou_score: 0.8314 - val_f1-score: 0.8979
Epoch 17/17
500/500 [==============================] - 599s 1s/step - loss: 0.0615 - iou_score: 0.8609 - f1-score: 0.9188 - val_loss: 0.0800 - val_iou_score: 0.8320 - val_f1-score: 0.8982